In [1]:
import os
import csv
import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')

# Ruta a la carpeta que contiene los archivos nii.gz
# data_dir = os.path.join('/content/drive', 'My Drive', 'UCR', '2-2024', 'InvCC', 'Investigación CC', 'Datasets', 'preprocessed')
data_dir = os.path.join('preprocessed')

In [2]:
# Leer archivo de etiquetas fenotípicas
try:
    phenotypes_file = os.path.join(data_dir, 'adhd200_preprocessed_phenotypics.csv')
    phenotypes_df = pd.read_csv(phenotypes_file)

    # Verificar que las columnas necesarias estén presentes
    required_columns = ['ScanDir ID', 'DX']
    if not all(col in phenotypes_df.columns for col in required_columns):
        raise ValueError("El archivo CSV no contiene las columnas requeridas")

    # Asegurarse de que los IDs tengan 7 dígitos
    phenotypes_df['ScanDir ID'] = phenotypes_df['ScanDir ID'].apply(lambda x: str(x).zfill(7))

    # Crear un diccionario de fenotipos
    phenotypes = phenotypes_df.set_index('ScanDir ID')['DX'].to_dict()
except FileNotFoundError:
    print("El archivo de fenotipos no se encontró")
except pd.errors.EmptyDataError:
    print("El archivo de fenotipos está vacío")
except pd.errors.ParserError as e:
    print(f"Error al parsear el archivo de fenotipos: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")

In [3]:
print(phenotypes)

{'2371032': '0', '2026113': '1', '3434578': '0', '8628223': '0', '1623716': '1', '1594156': '0', '2930625': '0', '3154996': '3', '3160561': '1', '4275075': '3', '2917777': '0', '2558999': '0', '2740232': '0', '1018959': '0', '2572285': '0', '1962503': '0', '1842819': '0', '1873761': '1', '3103809': '0', '3699991': '0', '1535233': '0', '4154182': '0', '6453038': '1', '3486975': '0', '3912996': '0', '1686265': '0', '2014113': '1', '1996183': '3', '7129258': '0', '1779922': '0', '3310328': '2', '2618929': '0', '1652369': '0', '4104523': '0', '8083695': '0', '6346605': '0', '9922944': '0', '1638334': '0', '1692275': '0', '2641332': '0', '8263351': '0', '3884955': '0', '2344857': '0', '2822304': '0', '8658218': '0', '3119327': '3', '2299519': '1', '3813783': '0', '2640795': '0', '4601682': '0', '3972472': '0', '2018106': '0', '2104012': '0', '5216908': '0', '1019436': '3', '8432725': '0', '1043241': '0', '2554127': '0', '3713230': '0', '3902469': '0', '3972956': '0', '7415617': '0', '351902

In [4]:
# Buscar archivos .nii.gz recursivamente solo en carpetas que se llamen 'fmri'
for root, dirs, files in os.walk(data_dir):
    if os.path.basename(root) == 'fmri':
        for file in files:
            if file.endswith('.nii.gz'):
                # Extraer el ID del sujeto del nombre del archivo
                filename = os.path.basename(file)
                parts = filename.split('_')
                subject_id = parts[2]

                # Obtener la etiqueta fenotípica
                diagnosis = phenotypes.get(subject_id)

                if diagnosis == '0':
                    new_filename = f"{subject_id}_TDC_session_{parts[4]}_{parts[5]}"
                    os.rename(os.path.join(root, file), os.path.join(data_dir, 'TDC', new_filename))
                elif diagnosis == '1' or diagnosis == '2' or diagnosis == '3':
                    new_filename = f"{subject_id}_ADHD_session_{parts[4]}_{parts[5]}"
                    os.rename(os.path.join(root, file), os.path.join(data_dir, 'ADHD', new_filename))